<a href="https://colab.research.google.com/github/jana0601/AA_Summer-school-LMMS/blob/main/Data-Driven%20Modeling%20of%20Dynamical%20Systems/LabSession_DoubleWell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.linalg as scl

import pyemma.coordinates as pco
import pyemma.msm as pmsm
import pyemma.plots as mplt

from msmtools.analysis.dense.pcca import _pcca_connected_isa

In this notebook, we will apply EDMD algorithm to analyze a diffusion in a two-well potential
$$ \mathrm{d}X_t = - V'(X_t) \mathrm{d}t + \sigma(X_t) \mathrm{d}W_t $$
$$ V(x) = (x^2 - 1)^2. $$

### Simulation and Evolution of Densities
Let's repeat what we did for the previous example: we set up the numerical integrator, and then have a look at the evolution of probability distributions with time.

In [ ]:
# Euler scheme for gradient flow SDE:
def Euler_Scheme(x0, sigma, dt, m, dV):
    # Prepare output:
    y = np.zeros(m)
    y[0] = x0
    # Initialize at x0:
    x = x0
    # Integrate:
    for kk in range(1, m):
        # Update:
        xn = x - dV(x) * dt + np.sqrt(dt) * sigma * np.random.randn(1)
        # Update current state:
        y[kk] = xn
        x = xn
    return y

Use the above function to produce 1000 simulations, each comprising discrete 100 steps, at integration time step 1e-2, starting from uniformly drawn initial conditions in $[-1.5, 1.5]$. Produce a histogram of the data after $[5, 10, 20, 50, 100]$ steps.

In [ ]:
# Settings:
m = 100
dt = 1e-2
ntraj = 1000

# Derivative of the energy:
dV = lambda x: 4*x*(x**2 - 1)
# Diffusion:
sigma = 1.0

# Produce simulation data:
X = np.zeros((ntraj, m))



In [ ]:
# Time instances to be used for histogramming:
t_vec = np.array([5, 10, 20, 50, 100])
# Bins for histogram:
xe = np.arange(-2.5, 2.51, 0.1)
xc = 0.5 * (xe[1:] + xe[:-1])

# Histogram the data at different time instances:
plt.figure()

qq = 0
for t in t_vec:

plt.xlabel("x", fontsize=12)
plt.tick_params(labelsize=12)
plt.legend(loc=2)
plt.title("Time Evolution of Distribution")

### Estimating the Koopman Operator

Produce 10,000 pairs $(x_l, y_l)$ by drawing $x_l$ from the invariant measure of our SDE. To do so, first draw initial conditions from a uniform distribution in $[-1.5, 1.5]$, and let them equilibrate by running the dynamics for 50 discrete steps. Use the terminal conditions of these simulations as $x_l$. Compute each $y_l$ by running the dynamics again over time $t = 0.1$ (just 10 discrete time steps). 

Then, estimate the Koopman matrix for a Gaussian basis with equidistant centers in $[-1.2, 1.2]$, with bandwidth $0.5$

In [ ]:
# Produce the data:
m = 10000
x = np.zeros(m)
y = np.zeros(m)
# Generate equilibrated initial conditions:

# Perform actual simulations:

# Define Gaussian basis set:
mu_vec = np.arange(-1.2, 1.21, 0.2)
n = mu_vec.shape[0]
sig_psi = 0.5

psi = 
# Compute Koopman matrix:
K = 

Compute the first few eigenvalues of the Koopman matrix. Then, use the spectral mapping theorem to project eigenvalues at lag times $[0.1, 0.2, ..., 2.0]$.

In [ ]:
# Diagonalize K:
d, V = scl.eig(K)
# Sort eigenvalues and eigenvectors:
ind = np.argsort(d)[::-1]
d = d[ind]
V = V[:, ind]

# Plot eigenvalues at multiple lag times:
lags = nsteps * np.arange(1, 21)
plt.figure()
for k in range(1, 3):
    plt.plot(dt*lags, d[k]**(lags / nsteps), "o")
plt.xlabel(r"$t$")
plt.ylabel(r"$\lambda_k(t)$")
plt.title("Eigenvalues predicted by Koopman matrix")

Infer the implied relaxation timescales from the eigenvalues of your Koopman matrix. These are defined as
$$ t_k = -\frac{t}{\log(\lambda_k(t))}$$

In [ ]:
# Determine timescales:
print("Implied timescales: ")
its = 
print(its)

### Finding Structure based on the Koopman matrix

We will now use the Koopman matrix to identify long-lived parts of the state space. 

In [ ]:
# Evaluate the first two eigenfunctions at all data points:
PX = np.zeros((n, m))
for l in range(m):
    PX[:, l] = psi(x[l])
VX = np.dot(V[:, :2].T, PX)
VX[0, :] = np.mean(VX[0, :])

In [ ]:
# Apply PCCA:
chi, A = _pcca_connected_isa(VX.T, n_clusters=2)

In [ ]:
# Evaluate membership functions on a grid:
xp = np.arange(-1.5, 1.51, 0.1)
pxp = np.zeros((n, xp.shape[0]))
for l in range(xp.shape[0]):
    pxp[:, l] = psi(xp[l])
chi_p = np.dot(A.T, np.dot(V[:, :2].T, pxp))

In [ ]:
for k in range(2):
    plt.plot(xp, chi_p[k, :], linewidth=2.5, label="State %d"%k)
plt.legend(loc=1)
plt.xlabel("x")
plt.title("Memberships for Macrostates")

### Alternative: MSM Construction

In [ ]:
# Generate a single long simulation:
xlong = Euler_Scheme(0.0, sigma, dt, 50000, dV)
plt.plot(xlong)

In [ ]:
# Assign each time step to a box:
dtraj = pco.assign_to_centers(xlong, centers=xc[:, None])

In [ ]:
# Complete implied timescales for a series of lag times:
its = pmsm.timescales_msm(dtraj, lags=nsteps * np.arange(1, 21), nits=4)

In [ ]:
# Plot the result:
mplt.plot_implied_timescales(its, linewidth=2.5, dt=dt, marker="o")